In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [16]:
scor_density = pd.read_csv("CREMP_SCOR_Summaries_2023_Density.csv")
octo_density = pd.read_csv("CREMP_OCTO_Summaries_2023_Density.csv")
scor_condition = pd.read_csv("CREMP_SCOR_Summaries_2023_ConditionCounts.csv")
temperature = pd.read_csv("CREMP_Temperatures_2023.csv")
stations = pd.read_csv("CREMP_Stations_2023.csv")
species_richness = pd.read_csv("CREMP_SCOR_Summaries_2023_LTA.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'CREMP_SCOR_Summaries_2023_Density.csv'

In [3]:
import os

file_path = r"C:\Users\sajiy\OneDrive\Desktop\gec project\corals\datasets\CREMP_OCTO_Summaries_2023_Density.xls"

if os.path.exists(file_path):
    print("File exists!")
else:
    print("File does NOT exist.")

File does NOT exist.


In [4]:
scor_density = pd.merge(scor_density, stations[['SiteID', 'Region']], on='SiteID', how='left')
temperature = pd.merge(temperature, stations[['SiteID', 'Region']], on='SiteID', how='left')
species_richness = pd.merge(species_richness, stations[['SiteID', 'Region']], on='SiteID', how='left')

NameError: name 'scor_density' is not defined

In [ ]:
species_cols = scor_density.columns[7:-2]  
scor_density['Density'] = scor_density[species_cols].sum(axis=1)

In [ ]:
density_yearly = scor_density.groupby('Year')['Density'].mean().reset_index()

In [ ]:
plt.figure(figsize=(10,6))
sns.lineplot(data=density_yearly, x='Year', y='Density', marker='o')
plt.title("Stony Coral Density Over Years")
plt.xlabel("Year")
plt.ylabel("Average Density")
plt.grid(True)
plt.show()


In [ ]:
avg_temp = temperature.groupby('Year')['TempC'].mean().reset_index()
avg_density = scor_density.groupby('Year')['Density'].mean().reset_index()
merged_temp_density = pd.merge(avg_density, avg_temp, on='Year')

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(data=merged_temp_density, x='TempC', y='Density', s=100, color='blue')
sns.regplot(data=merged_temp_density, x='TempC', y='Density', scatter=False, color='red')
plt.title("Temperature vs Coral Density Over Years")
plt.xlabel("Average Temperature ")
plt.ylabel("Average Coral Density")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
region_density = scor_density.groupby('Region')['Density'].mean().sort_values(ascending=False)

In [5]:
plt.figure(figsize=(12,6))
plt.bar(region_density.index, region_density.values, color='skyblue')
plt.title("Average Coral Density by Region")
plt.xlabel("Region")
plt.ylabel("Average Coral Density")
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
plt.show()

NameError: name 'region_density' is not defined

<Figure size 1200x600 with 0 Axes>

In [6]:
coral_species_cols = species_richness.columns[7:-1]  
species_richness['SpeciesRichness'] = (species_richness[coral_species_cols] > 0).sum(axis=1)
richness_yearly = species_richness.groupby('Year')['SpeciesRichness'].mean().reset_index()

NameError: name 'species_richness' is not defined

In [7]:
plt.figure(figsize=(10,6))
sns.lineplot(data=richness_yearly, x='Year', y='SpeciesRichness', marker='o', color='green')
plt.title("Species Richness Over Time")
plt.xlabel("Year")
plt.ylabel("Average Species Richness")
plt.grid(True)
plt.tight_layout()
plt.show()

NameError: name 'richness_yearly' is not defined

<Figure size 1000x600 with 0 Axes>

In [8]:
if all(col in scor_density.columns for col in ['SiteID', 'Year', 'Density']):
    if 'SpeciesRichness' not in species_richness.columns:
        species_richness['SpeciesRichness'] = species_richness[richness_columns].gt(0).sum(axis=1)

    merged_data = pd.merge(scor_density, species_richness[['SiteID', 'Year', 'SpeciesRichness']], on=['SiteID', 'Year'], how='left')
    avg_temp_station = temperature.groupby(['SiteID', 'Year'])['TempC'].mean().reset_index()
    merged_data = pd.merge(merged_data, avg_temp_station, on=['SiteID', 'Year'], how='left')
    merged_data.dropna(subset=['Density', 'SpeciesRichness', 'TempC', 'Region'], inplace=True)

    # Label encoding for Region
    le = LabelEncoder()
    merged_data['Region_encoded'] = le.fit_transform(merged_data['Region'])

    # Features and target
    X = merged_data[['Year', 'SpeciesRichness', 'TempC', 'Region_encoded']]
    y = merged_data['Density']

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print("Training Data Shape:", X_train.shape)
    print("Testing Data Shape:", X_test.shape)

    # Model training
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Predict
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("R^2 Score:", r2)
print("RMSE:", rmse)

NameError: name 'scor_density' is not defined

In [9]:
#import pandas as pd

comparison_df = pd.DataFrame({
    'Actual': y_test.values,
    'Predicted': y_pred
})

print(comparison_df.head(10))

NameError: name 'y_test' is not defined

In [10]:
#import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.scatter(y_test, y_pred, color='pink')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'b--')  # ideal line
plt.xlabel('Actual Coral Density')
plt.ylabel('Predicted Coral Density')
plt.title('Actual vs Predicted Coral Density')
plt.grid(True)
plt.show()

NameError: name 'y_test' is not defined

<Figure size 1000x500 with 0 Axes>

In [11]:
def predict_future_coral_density():
    try:
        future_year = int(input("Enter Year (e.g., 2026): "))
        future_richness = float(input("Enter Species Richness (e.g., 35): "))
        future_temp = float(input("Enter Temperature (°C, e.g., 28.5): "))
        region_input = input(f"Enter Region {list(region_dict.keys())}: ")

        if region_input not in region_dict:
            print("❌ Invalid region! Try one of:", list(region_dict.keys()))
            return

        future_region_encoded = region_dict[region_input]

        # Use DataFrame for feature names
        future_input = pd.DataFrame([{
            'Year': future_year,
            'SpeciesRichness': future_richness,
            'TempC': future_temp,
            'Region_encoded': future_region_encoded
        }])

        predicted_density = model.predict(future_input)
        print(f"🌊 Predicted Coral Density in {future_year}: {predicted_density[0]:.2f} colonies/m²")
    
    except ValueError:
        print("❌ Invalid input. Please enter correct numeric values.")
predict_future_coral_density()

Enter Year (e.g., 2026):  2078
Enter Species Richness (e.g., 35):  34
Enter Temperature (°C, e.g., 28.5):  23.8


NameError: name 'region_dict' is not defined

In [ ]:
Create a Flask App (app.py):
- Import Flask and necessary libraries.
- Set up routes for each page (e.g., /, /dashboard, /predict).
- Load and process your datasets (use the scripts you've developed).
- Integrate your predictive model for dynamic predictions.

Example:
from flask import Flask, render_template, request
import pandas as pd
import joblib

app = Flask(__name__)

# Load model
model = joblib.load('models/coral_density_model.pkl')

@app.route('/')
def home():
    return render_template('home.html')

@app.route('/predict', methods=['POST'])
def predict():
    # Process user input and make prediction
    # Return results to user
    pass

if __name__ == '__main__':
    app.run(debug=True)